#Actividad 7: Regresión Logística

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [ ]:
df = pd.read_csv('cuentas_credicel.csv', encoding='latin-1')
df.head(5)

<ipython-input-1380-c71743e22efc>:1: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('cuentas_credicel.csv', encoding='latin-1')


,folio,tag,folio_solicitud,fecha,marca,modelo,plazo,precio,enganche,descuento,...,porc_eng,limite_credito,semana_actual,cp_cliente,edad_cliente,cd_cliente,edo_cliente,cd_venta,edo_venta,curp
0,3,HDNAWQ3A,30,12/11/21 19:00,ZTE,Blade A3 2020 RO B,26S,1949.0,780.0,0.0,...,40.02,5000,26,NaN,23.0,0,0,Tantoyuca,Veracruz,AEVM980413MVZNDR02
1,4,M418YNR4,33,12/11/21 19:15,MOTOROLA,LTE XT2097-12 E7I POWER Azul,13S,2999.0,1050.0,0.0,...,35.01,6000,13,NaN,60.0,PUEBLA,PUE.,Heroica Puebla de Zaragoza,Puebla,ZALC610119MPLPRR07
2,5,SZ7V3NZT,37,13/11/21 14:13,ZTE,BLADE A3 2020 Gris,13S,1959.0,490.0,0.0,...,25.01,0,74,NaN,58.0,TANTOYUCA,VER.,0,0,0
3,6,9I08UIQZ,38,13/11/21 15:45,ZTE,BLADE A3 2020 Gris,13S,1959.0,490.0,0.0,...,25.01,7000,13,NaN,42.0,AMXTLAN PUE,NaN,Zacatlan,Puebla,SAMJ790115MPLNRS02
4,7,JMF4LKYU,44,13/11/21 16:10,MOTOROLA,G10 B,39S,4299.0,1075.0,0.0,...,25.01,7000,74,NaN,33.0,TANTOYUCA,VER.,Tantoyuca,Veracruz,TEHF880707HVZRRR07


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22735 entries, 0 to 22734
Data columns (total 39 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   folio               22735 non-null  int64  
 1   tag                 22735 non-null  object 
 2   folio_solicitud     22735 non-null  int64  
 3   fecha               22735 non-null  object 
 4   marca               22735 non-null  object 
 5   modelo              22735 non-null  object 
 6   plazo               22735 non-null  object 
 7   precio              22735 non-null  float64
 8   enganche            22735 non-null  float64
 9   descuento           22735 non-null  float64
 10  semana              22735 non-null  int64  
 11  monto_financiado    22735 non-null  float64
 12  costo_total         22735 non-null  int64  
 13  monto_accesorios    22735 non-null  float64
 14  agente_venta        22735 non-null  object 
 15  dis_venta           22735 non-null  object 
 16  stat

##Valores nulos

In [ ]:
df.isnull().sum()

folio                    0
tag                      0
folio_solicitud          0
fecha                    0
marca                    0
modelo                   0
plazo                    0
precio                   0
enganche                 0
descuento                0
semana                   0
monto_financiado         0
costo_total              0
monto_accesorios         0
agente_venta             0
dis_venta                0
status                   0
fraude                   0
empresa                 13
inversion                0
pagos_realizados         0
reautorizacion           0
fecha_ultimo_pago     3356
fecha_pago_proximo    3356
status_cuenta         3314
puntos                   0
riesgo                   0
score_buro               0
razones_buro          7340
porc_eng                 0
limite_credito           0
semana_actual            0
cp_cliente            7243
edad_cliente             9
cd_cliente               0
edo_cliente              8
cd_venta                 0
e

In [ ]:
df['limite_credito'] = df['limite_credito'].replace('SD', 0)
df['limite_credito'] = df['limite_credito'].astype(float)

In [ ]:
df1 =df.fillna(method="bfill")

In [ ]:
df2 = df1.fillna(method="ffill")

In [ ]:
df2.isnull().sum()

folio                 0
tag                   0
folio_solicitud       0
fecha                 0
marca                 0
modelo                0
plazo                 0
precio                0
enganche              0
descuento             0
semana                0
monto_financiado      0
costo_total           0
monto_accesorios      0
agente_venta          0
dis_venta             0
status                0
fraude                0
empresa               0
inversion             0
pagos_realizados      0
reautorizacion        0
fecha_ultimo_pago     0
fecha_pago_proximo    0
status_cuenta         0
puntos                0
riesgo                0
score_buro            0
razones_buro          0
porc_eng              0
limite_credito        0
semana_actual         0
cp_cliente            0
edad_cliente          0
cd_cliente            0
edo_cliente           0
cd_venta              0
edo_venta             0
curp                  0
dtype: int64

##Datos atípicos


In [ ]:
numeric_columns = df2.select_dtypes(include=['int64','float64'])
dfNumerico = numeric_columns
categoric_columns = df2.select_dtypes(include=['object'])
dfCategorico = categoric_columns

In [ ]:
y=dfNumerico
Limite_Superior= y.mean() + 3* y.std()
Limite_Inferior= y.mean() - 3 * y.std()
print("Limite Superior Permitido", Limite_Superior)
print("Limite Inferior Permitido", Limite_Inferior)

Limite Superior Permitido folio                31323.107698
folio_solicitud     127723.906961
precio                8081.558082
enganche              2356.496845
descuento              478.215020
semana                 541.287392
monto_financiado      6366.317810
costo_total          12857.588246
monto_accesorios       191.260880
status                   2.611178
fraude                   0.864131
inversion                1.511599
pagos_realizados        39.306132
reautorizacion           0.578086
puntos                  48.354233
riesgo                  58.456322
score_buro            1142.021000
porc_eng                45.711306
limite_credito       12331.577603
semana_actual           64.380898
edad_cliente           422.314014
dtype: float64
Limite Inferior Permitido folio               -8301.055796
folio_solicitud    -36023.715538
precio                -87.484900
enganche             -503.622066
descuento            -321.263535
semana                -65.792692
monto_financiado     

In [ ]:
dfNumerico = numeric_columns[(y<=Limite_Superior)&(y>=Limite_Inferior)]
dfNumerico.head(20)

,folio,folio_solicitud,precio,enganche,descuento,semana,monto_financiado,costo_total,monto_accesorios,status,...,inversion,pagos_realizados,reautorizacion,puntos,riesgo,score_buro,porc_eng,limite_credito,semana_actual,edad_cliente
0,3,30,1949.0,780.0,0.0,81.0,1169.0,2106.0,0.0,2.0,...,0,0.0,0.0,0.0,0.0,0,40.02,5000.0,26.0,23.0
1,4,33,2999.0,1050.0,0.0,225.0,1949.0,2925.0,0.0,2.0,...,0,0.0,0.0,0.0,0.0,0,35.01,6000.0,13.0,60.0
2,5,37,1959.0,490.0,0.0,170.0,1469.0,2210.0,0.0,NaN,...,0,0.0,0.0,0.0,0.0,0,25.01,0.0,NaN,58.0
3,6,38,1959.0,490.0,0.0,170.0,1469.0,2210.0,0.0,2.0,...,0,0.0,0.0,0.0,0.0,0,25.01,7000.0,13.0,42.0
4,7,44,4299.0,1075.0,0.0,178.0,3224.0,6942.0,0.0,1.0,...,0,0.0,0.0,0.0,0.0,0,25.01,7000.0,NaN,33.0
5,8,49,2999.0,750.0,0.0,156.0,2249.0,4056.0,0.0,2.0,...,0,0.0,0.0,0.0,0.0,0,25.01,7000.0,26.0,38.0
6,9,50,2999.0,900.0,0.0,145.0,2099.0,3770.0,0.0,2.0,...,0,0.0,0.0,0.0,0.0,0,30.01,6000.0,26.0,21.0
7,10,54,3999.0,1500.0,0.0,288.0,2499.0,3744.0,0.0,2.0,...,0,0.0,0.0,0.0,0.0,0,37.51,7000.0,13.0,45.0
8,11,59,2889.0,1000.0,0.0,131.0,1889.0,3406.0,0.0,2.0,...,0,0.0,0.0,0.0,0.0,0,34.61,7000.0,26.0,36.0
9,12,62,2999.0,750.0,0.0,156.0,2249.0,4056.0,0.0,1.0,...,0,15.0,0.0,0.0,0.0,0,25.01,7000.0,NaN,47.0


In [ ]:
dfNumerico.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22735 entries, 0 to 22734
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   folio             22735 non-null  int64  
 1   folio_solicitud   22735 non-null  int64  
 2   precio            22625 non-null  float64
 3   enganche          22443 non-null  float64
 4   descuento         22718 non-null  float64
 5   semana            22373 non-null  float64
 6   monto_financiado  22553 non-null  float64
 7   costo_total       22445 non-null  float64
 8   monto_accesorios  22118 non-null  float64
 9   status            22581 non-null  float64
 10  fraude            21034 non-null  float64
 11  inversion         22735 non-null  int64  
 12  pagos_realizados  22551 non-null  float64
 13  reautorizacion    21961 non-null  float64
 14  puntos            22732 non-null  float64
 15  riesgo            22670 non-null  float64
 16  score_buro        22735 non-null  int64 

In [ ]:
dfCategorico.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22735 entries, 0 to 22734
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   tag                 22735 non-null  object
 1   fecha               22735 non-null  object
 2   marca               22735 non-null  object
 3   modelo              22735 non-null  object
 4   plazo               22735 non-null  object
 5   agente_venta        22735 non-null  object
 6   dis_venta           22735 non-null  object
 7   empresa             22735 non-null  object
 8   fecha_ultimo_pago   22735 non-null  object
 9   fecha_pago_proximo  22735 non-null  object
 10  status_cuenta       22735 non-null  object
 11  razones_buro        22735 non-null  object
 12  cp_cliente          22735 non-null  object
 13  cd_cliente          22735 non-null  object
 14  edo_cliente         22735 non-null  object
 15  cd_venta            22735 non-null  object
 16  edo_venta           22

In [ ]:
dfCategorico.isnull().sum()

tag                   0
fecha                 0
marca                 0
modelo                0
plazo                 0
agente_venta          0
dis_venta             0
empresa               0
fecha_ultimo_pago     0
fecha_pago_proximo    0
status_cuenta         0
razones_buro          0
cp_cliente            0
cd_cliente            0
edo_cliente           0
cd_venta              0
edo_venta             0
curp                  0
dtype: int64

In [ ]:
df3 = pd.concat([dfCategorico, dfNumerico], axis = 1)

In [ ]:
df3.isnull().sum()

tag                      0
fecha                    0
marca                    0
modelo                   0
plazo                    0
agente_venta             0
dis_venta                0
empresa                  0
fecha_ultimo_pago        0
fecha_pago_proximo       0
status_cuenta            0
razones_buro             0
cp_cliente               0
cd_cliente               0
edo_cliente              0
cd_venta                 0
edo_venta                0
curp                     0
folio                    0
folio_solicitud          0
precio                 110
enganche               292
descuento               17
semana                 362
monto_financiado       182
costo_total            290
monto_accesorios       617
status                 154
fraude                1701
inversion                0
pagos_realizados       184
reautorizacion         774
puntos                   3
riesgo                  65
score_buro               0
porc_eng               275
limite_credito           7
s

In [ ]:
df4 =df3.fillna(method="bfill")

In [ ]:
df5 =df4.fillna(method="ffill")

In [ ]:
df5.isnull().sum()

tag                   0
fecha                 0
marca                 0
modelo                0
plazo                 0
agente_venta          0
dis_venta             0
empresa               0
fecha_ultimo_pago     0
fecha_pago_proximo    0
status_cuenta         0
razones_buro          0
cp_cliente            0
cd_cliente            0
edo_cliente           0
cd_venta              0
edo_venta             0
curp                  0
folio                 0
folio_solicitud       0
precio                0
enganche              0
descuento             0
semana                0
monto_financiado      0
costo_total           0
monto_accesorios      0
status                0
fraude                0
inversion             0
pagos_realizados      0
reautorizacion        0
puntos                0
riesgo                0
score_buro            0
porc_eng              0
limite_credito        0
semana_actual         0
edad_cliente          0
dtype: int64

In [ ]:
df_coef1 = pd.DataFrame(columns=['Variable', 'Analisis', 'Precision', 'Exactitud','Sensibilidad'])

##Análisis 1

In [ ]:
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= df5[['descuento', 'precio', 'enganche']]
Var_Dep= df5['marca']

In [ ]:
#Redefinimos las variables
X= Vars_Indep
y= Var_Dep

In [ ]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)

In [ ]:
#Se escalan todos los datos
escalar = StandardScaler()

In [ ]:
#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [ ]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [ ]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['ZTE', 'Samsung', 'Samsung', ..., 'Samsung', 'Samsung', 'Samsung'],
      dtype=object)

In [ ]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[   0    0    0    0    0    0    0    0    0    0    1    0    0    0
     0   21    0    0    0    0    0   34    0]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0   11    0    0    0    0    0   13    0]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    1    0]
 [   0    0    0    0    0    0    0    0    1    0    0    0    0    0
     1   44    0    0    0    0    0   50    0]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     6   90    0    2    0    0    0   24    0]
 [   0    0    0    0    0    0    0    0    0    0    1    0    0    0
     0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    6    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    4    0    0    0    0    0   20    0]
 [   0    0    0   

In [ ]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score
precision1 = precision_score(y_test, y_pred, average="macro", pos_label="yes")
print('Precisión del modelo:')
print(precision1)

Precisión del modelo:
0.08669775308455425


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'yes') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud1 = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud1)

Exactitud del modelo:
0.4417240873772174


In [ ]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad1 = recall_score(y_test, y_pred, average="macro", pos_label="yes")
print('Sensibilidad del modelo:')
print(sensibilidad1)

Sensibilidad del modelo:
0.08262939525426954


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'yes') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


In [ ]:
# el puntaje F1 que es una combinación entre la precisión y la sensibilidad, para esto importamos f1_score.
#Calculo el Puntaje F1 del modelo
from sklearn.metrics import f1_score
puntajef1 = f1_score(y_test, y_pred, average="macro", pos_label="yes")
print('Puntaje F1 del modelo:')
print(puntajef1)

Puntaje F1 del modelo:
0.06753151417075592


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'yes') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


In [ ]:
df_coef1.loc[0] = ['marca', '1', round(precision1,2), round(sensibilidad1,2), round(puntajef1,2) ]

##Análisis 2

In [ ]:
Vars_Indep= df5[['enganche', 'precio', 'descuento']]
Var_Dep= df5['puntos']

In [ ]:
X= Vars_Indep
y= Var_Dep

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)

In [ ]:
escalar = StandardScaler()

In [ ]:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [ ]:
algoritmo = LogisticRegression()

In [ ]:
algoritmo.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
y_pred = algoritmo.predict(X_test)
y_pred

array([12.,  0., 12., ..., 12., 13.,  0.])

In [ ]:
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
precision2 = precision_score(y_test, y_pred, average="macro")
print('Precisión del modelo:')
print(precision2)

Precisión del modelo:
0.022338006489035802


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
sensibilidad2 = recall_score(y_test, y_pred, average="macro")
print('Sensibilidad del modelo:')
print(sensibilidad2)

Sensibilidad del modelo:
0.028919965739175185


In [ ]:
puntajef2 = f1_score(y_test, y_pred, average="macro")
print('Puntaje F1 del modelo:')
print(puntajef2)

Puntaje F1 del modelo:
0.023241077558314758


In [ ]:
df_coef1.loc[1] = ['puntos', '2', round(precision2,2), round(sensibilidad2,2), round(puntajef2,2) ]

##Análisis 3

In [ ]:
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= df5[['precio', 'semana', 'edad_cliente']]
Var_Dep= df5['status_cuenta']

In [ ]:
#Redefinimos las variables
X= Vars_Indep
y= Var_Dep

In [ ]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)

In [ ]:
#Se escalan todos los datos
escalar = StandardScaler()

In [ ]:
#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [ ]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [ ]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [ ]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['Corriente', 'Corriente', 'Corriente', ..., 'Corriente',
       'Corriente', 'Corriente'], dtype=object)

In [ ]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[   0    0 1012    0    1]
 [   0    0   74    0    0]
 [   0    0 3289    0    6]
 [   0    0  619    0    0]
 [   0    0 1813    0    7]]


In [ ]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score
precision3 = precision_score(y_test, y_pred, average="macro", pos_label="yes")
print('Precisión del modelo:')
print(precision3)

Precisión del modelo:
0.19663581607169092


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'yes') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud3 = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud3)

Exactitud del modelo:
0.4832136050432488


In [ ]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad3 = recall_score(y_test, y_pred, average="macro", pos_label="yes")
print('Sensibilidad del modelo:')
print(sensibilidad3)

Sensibilidad del modelo:
0.20040504260534608


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'yes') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


In [ ]:
# el puntaje F1 que es una combinación entre la precisión y la sensibilidad, para esto importamos f1_score.
#Calculo el Puntaje F1 del modelo
from sklearn.metrics import f1_score
puntajef3 = f1_score(y_test, y_pred, average="macro", pos_label="yes")
print('Puntaje F1 del modelo:')
print(puntajef3)

Puntaje F1 del modelo:
0.13175835485679654


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'yes') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


In [ ]:
df_coef1.loc[2] = ['status_cuenta', '3', round(precision3,2), round(sensibilidad3,2), round(puntajef3,2) ]

## Analisis 4

In [ ]:
Vars_Indep= df5[['enganche', 'precio', 'descuento']]
Var_Dep= df5['costo_total']

In [ ]:
X= Vars_Indep
y= Var_Dep

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)

In [ ]:
escalar = StandardScaler()

In [ ]:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [ ]:
algoritmo = LogisticRegression()

In [ ]:
algoritmo.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
y_pred = algoritmo.predict(X_test)
y_pred

array([6188., 4108., 4108., ..., 4108., 4108., 7202.])

In [ ]:
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
precision4 = precision_score(y_test, y_pred, average="macro")
print('Precisión del modelo:')
print(precision4)

Precisión del modelo:
0.0016664314215187857


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
sensibilidad4 = recall_score(y_test, y_pred, average="macro")
print('Sensibilidad del modelo:')
print(sensibilidad4)

Sensibilidad del modelo:
0.0077562663477017965


In [ ]:
puntajef4 = f1_score(y_test, y_pred, average="macro")
print('Puntaje F1 del modelo:')
print(puntajef4)

Puntaje F1 del modelo:
0.002127709952727934


In [ ]:
df_coef1.loc[3] = ['costo_total', '4', round(precision4,2), round(sensibilidad4,2), round(puntajef4,2) ]

##Análisis 5

In [ ]:
Vars_Indep= df5[['porc_eng', 'precio', 'descuento']]
Var_Dep= df5['limite_credito']

In [ ]:
X= Vars_Indep
y= Var_Dep

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)

In [ ]:
escalar = StandardScaler()

In [ ]:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [ ]:
algoritmo = LogisticRegression()

In [ ]:
algoritmo.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
y_pred = algoritmo.predict(X_test)
y_pred

array([ 4000.,  7000., 10000., ..., 10000.,  4000., 10000.])

In [ ]:
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[  0   0   1   0   0   0   0   3  25   0   0   0   0   0   0   7   0   0
    0   0   0   6   0   0]
 [  0   0   0   0   0   0   0   1   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0]
 [  0   0   7   0   0   0   0  34  13   0   0   0   0   0   0   1   0   0
    0   0   0   1   0   0]
 [  0   0   0   0   0   0   0  21  14   0   0   0   0   0   0   1   0   0
    0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   2   7   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0]
 [  0   0   2   0   0   0   0  29  36   0   0   0   0   0   0  12   0   0
    0   0   0   5   0   0]
 [  0   0   0   0   0   0   0   1   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0]
 [  0   0   0   0   0   0   0 179 363   0   0   0   0   0   0  15   0   0
    0   0   0   2   0   0]
 [  0   0   2   0   0   0   0  76 913   0   0   0   0   0   0 356   0   0
    0   0   0   4   0   0]
 [  0   0   0   0   0   0   0   7 432   0   0   0   0   0   0  37   0 

In [ ]:
precision5 = precision_score(y_test, y_pred, average="weighted")
print('Precisión del modelo:')
print(precision5)

Precisión del modelo:
0.17081089751699477


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
sensibilidad5 = recall_score(y_test, y_pred, average="weighted")
print('Sensibilidad del modelo:')
print(sensibilidad5)

Sensibilidad del modelo:
0.2918926843571324


In [ ]:
puntajef5 = f1_score(y_test, y_pred, average="weighted")
print('Puntaje F1 del modelo:')
print(puntajef5)

Puntaje F1 del modelo:
0.2028999193420863


In [ ]:
df_coef1.loc[4] = ['limite_credito', '5', round(precision5,2), round(sensibilidad5,2), round(puntajef5,2) ]

##Análisis 6

In [ ]:
Vars_Indep= df5[['enganche', 'precio', 'descuento']]
Var_Dep= df5['semana_actual']

In [ ]:
X= Vars_Indep
y= Var_Dep

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)

In [ ]:
escalar = StandardScaler()

In [ ]:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [ ]:
algoritmo = LogisticRegression()

In [ ]:
algoritmo.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
y_pred = algoritmo.predict(X_test)
y_pred

array([26., 17., 26., ..., 13., 13., 13.])

In [ ]:
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
precision6 = precision_score(y_test, y_pred, average="micro")
print('Precisión del modelo:')
print(precision6)

Precisión del modelo:
0.15085764550652397


In [ ]:
sensibilidad6 = recall_score(y_test, y_pred, average="weighted")
print('Sensibilidad del modelo:')
print(sensibilidad6)

Sensibilidad del modelo:
0.15085764550652397


In [ ]:
puntajef6 = f1_score(y_test, y_pred, average="weighted")
print('Puntaje F1 del modelo:')
print(puntajef6)

Puntaje F1 del modelo:
0.06084483897673403


In [ ]:
df_coef1.loc[5] = ['semana_actual', '6', round(precision6,2), round(sensibilidad6,2), round(puntajef6,2) ]

##Análisis 7

In [ ]:
Vars_Indep= df5[['enganche', 'precio', 'descuento']]
Var_Dep= df5['pagos_realizados']

In [ ]:
X= Vars_Indep
y= Var_Dep

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)

In [ ]:
escalar = StandardScaler()

In [ ]:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [ ]:
algoritmo = LogisticRegression()

In [ ]:
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [ ]:
y_pred = algoritmo.predict(X_test)
y_pred

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[1279    0    0 ...    0    0    0]
 [ 315    0    0 ...    0    0    0]
 [ 268    0    0 ...    0    0    0]
 ...
 [  24    0    0 ...    0    0    0]
 [  21    0    0 ...    0    0    0]
 [  20    0    0 ...    0    0    0]]


In [ ]:
precision7 = precision_score(y_test, y_pred, average="macro")
print('Precisión del modelo:')
print(precision7)

Precisión del modelo:
0.011560486368954112


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
sensibilidad7 = recall_score(y_test, y_pred, average="macro")
print('Sensibilidad del modelo:')
print(sensibilidad7)

Sensibilidad del modelo:
0.027682511469953215


In [ ]:
puntajef7 = f1_score(y_test, y_pred, average="macro")
print('Puntaje F1 del modelo:')
print(puntajef7)

Puntaje F1 del modelo:
0.011825177901081453


In [ ]:
df_coef1.loc[6] = ['pagos_realizados', '7', round(precision7,2), round(sensibilidad7,2), round(puntajef7,2) ]

##Análisis 8

In [ ]:
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= df5[['monto_financiado', 'semana_actual', 'score_buro']]
Var_Dep= df5['edo_cliente']

In [ ]:
#Redefinimos las variables
X= Vars_Indep
y= Var_Dep

In [ ]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)

In [ ]:
#Se escalan todos los datos
escalar = StandardScaler()

In [ ]:
#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [ ]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [ ]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['PUE', 'PUE', 'PUE', ..., ' PUE.', ' PUE.', 'PUE'], dtype=object)

In [ ]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
precision8 = precision_score(y_test, y_pred, average="weighted", pos_label="yes")
print('Precisión del modelo:')
print(precision8)

Precisión del modelo:
0.12256503836823571


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'yes') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
exactitud8 = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud8)

Exactitud del modelo:
0.23867468113179885


In [ ]:
sensibilidad8 = recall_score(y_test, y_pred, average="weighted", pos_label="yes")
print('Sensibilidad del modelo:')
print(sensibilidad8)

Sensibilidad del modelo:
0.23867468113179885


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'yes') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


In [ ]:
puntajef8 = f1_score(y_test, y_pred, average="weighted", pos_label="yes")
print('Puntaje F1 del modelo:')
print(puntajef8)

Puntaje F1 del modelo:
0.13910779748218804


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'yes') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


In [ ]:
df_coef1.loc[7] = ['edo_cliente', '8', round(precision8,2), round(sensibilidad8,2), round(puntajef8,2) ]

##Análisis 9

In [ ]:
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= df5[['porc_eng', 'descuento', 'precio']]
Var_Dep= df5['modelo']

In [ ]:
#Redefinimos las variables
X= Vars_Indep
y= Var_Dep

In [ ]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)

In [ ]:
#Se escalan todos los datos
escalar = StandardScaler()

In [ ]:
#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [ ]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [ ]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['A03s', 'A03s', 'Moto E22i', ..., 'A03', 'A03s', 'A03'],
      dtype=object)

In [ ]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
precision9 = precision_score(y_test, y_pred, average="weighted", pos_label="yes")
print('Precisión del modelo:')
print(precision9)

Precisión del modelo:
0.12047682933920899


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'yes') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
exactitud9 = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud9)

Exactitud del modelo:
0.2358891658114646


In [ ]:

sensibilidad9 = recall_score(y_test, y_pred, average="macro", pos_label="yes")
print('Sensibilidad del modelo:')
print(sensibilidad9)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'yes') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


Sensibilidad del modelo:
0.021130771534821065


In [ ]:

puntajef9 = f1_score(y_test, y_pred, average="weighted", pos_label="yes")
print('Puntaje F1 del modelo:')
print(puntajef9)

Puntaje F1 del modelo:
0.14272204183823237


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'yes') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


In [ ]:
df_coef1.loc[8] = ['modelo', '9', round(precision9,2), round(sensibilidad9,2), round(puntajef9,2) ]

##Análisis 10

In [ ]:
Vars_Indep= df5[['score_buro', 'precio', 'riesgo']]
Var_Dep= df5['edad_cliente']

In [ ]:
X= Vars_Indep
y= Var_Dep

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)

In [ ]:
escalar = StandardScaler()

In [ ]:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [ ]:
algoritmo = LogisticRegression()

In [ ]:
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [ ]:
y_pred = algoritmo.predict(X_test)
y_pred

array([38., 24., 18., ..., 24., 18., 18.])

In [ ]:
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[100  26   2 ...   0   0   0]
 [ 89  47   2 ...   0   0   0]
 [ 81  39   1 ...   0   0   0]
 ...
 [  0   0   0 ...   0   0   0]
 [  0   2   0 ...   0   0   0]
 [  0   1   0 ...   0   0   0]]


In [ ]:
precision10 = precision_score(y_test, y_pred, average="micro")
print('Precisión del modelo:')
print(precision10)

Precisión del modelo:
0.0432487904999267


In [ ]:
sensibilidad10 = recall_score(y_test, y_pred, average="macro")
print('Sensibilidad del modelo:')
print(sensibilidad10)

Sensibilidad del modelo:
0.024701447367016852


In [ ]:
puntajef10 = f1_score(y_test, y_pred, average="weighted")
print('Puntaje F1 del modelo:')
print(puntajef10)

Puntaje F1 del modelo:
0.01627333915616254


In [ ]:
df_coef1.loc[9] = ['edad_cliente', '10', round(precision10,2), round(sensibilidad10,2), round(puntajef10,2) ]

##Tabla de coeficientes

In [ ]:
df_coef1.reset_index()


,index,Variable,Analisis,Precision,Exactitud,Sensibilidad
0,0,marca,1,0.09,0.08,0.07
1,1,puntos,2,0.02,0.03,0.02
2,2,status_cuenta,3,0.20,0.20,0.13
3,3,costo_total,4,0.00,0.01,0.00
4,4,limite_credito,5,0.17,0.29,0.20
5,5,semana_actual,6,0.15,0.15,0.06
6,6,pagos_realizados,7,0.01,0.03,0.01
7,7,edo_cliente,8,0.12,0.24,0.14
8,8,modelo,9,0.12,0.02,0.14
9,9,edad_cliente,10,0.04,0.02,0.02
